# Intra-Candle Tick Analysis Demo

This notebook demonstrates the **IntraBarZigzag** algorithm.
It loads sample tick data, processes it to identify deterministic swings based on intra-candle price paths, and visualizes the results.

### Objectives
1. Load high-frequency tick data.
2. Apply `IntraBarZigzag` algorithm to resolve High/Low timing sequences.
3. Visualize the "True Zigzag" points against raw tick data.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Add the 'src' directory to the system path to import the custom class
sys.path.append(os.path.abspath('../src'))

# Import the logic class
from intra_bar_zigzag import IntraBarZigzag

print("Libraries and class imported successfully.")

In [ ]:
# Path to the sample data (relative path)
file_path = "../data/sample_tick_data.csv"

# Load the data
# Assuming the CSV has an index or specific columns. Adjust depending on your CSV format.
try:
    tick_df = pd.read_csv(file_path)
    # Ensure 'Time (EET)' is datetime for proper plotting later
    if 'Time (EET)' in tick_df.columns:
        tick_df['Time (EET)'] = pd.to_datetime(tick_df['Time (EET)'])
    
    print(f"Data loaded: {len(tick_df)} rows")
    display(tick_df.head())

except FileNotFoundError:
    print("Error: Sample data not found. Please ensure '../data/sample_tick_data.csv' exists.")

In [ ]:
# Initialize the algorithm with 1-minute resampling logic
intraBz = IntraBarZigzag(tick_df, 1)

# Setup (formatting and resampling)
print("Running setup...")
intraBz.runSetup()

# Run the detection logic
print("Running detection...")
intraBz.runDetection()

print(f"Detection completed. Found {len(intraBz.swings)} swings.")

In [ ]:
# Convert swings list to a DataFrame for easier plotting
swings_df = pd.DataFrame(intraBz.swings)
swings_df['time'] = pd.to_datetime(swings_df['time'])

# Get the time range from the original tick_df for filtering
start_time = tick_df['Time (EET)'].min()
end_time = tick_df['Time (EET)'].max()

# Filter swings_df to include only swings within the tick_df time range
filtered_swings_df = swings_df[(swings_df['time'] >= start_time) & (swings_df['time'] <= end_time)].copy()

# Inspect the results
print("First 5 rows of identified swings:")
display(filtered_swings_df.head())

In [ ]:
# Create the plot
fig = go.Figure()

# Add the tick data trace (Raw Price)
fig.add_trace(go.Scattergl(
    x=tick_df['Time (EET)'], 
    y=tick_df['Bid'], 
    mode='lines', 
    name='Tick Price', 
    line=dict(color='darkblue', width=1)
))

# Add swing high markers
high_swings = filtered_swings_df[filtered_swings_df['type'] == 'high']
fig.add_trace(go.Scattergl(
    x=high_swings['time'],
    y=high_swings['price'],
    mode='markers',
    name='Swing Highs',
    marker=dict(
        color='red',
        size=10,
        symbol='arrow-down'
    )
))

# Add swing low markers
low_swings = filtered_swings_df[filtered_swings_df['type'] == 'low']
fig.add_trace(go.Scattergl(
    x=low_swings['time'],
    y=low_swings['price'],
    mode='markers',
    name='Swing Lows',
    marker=dict(
        color='green',
        size=10,
        symbol='arrow-up'
    )
))

# Update layout
fig.update_layout(
    title='Intra-Bar Zigzag Analysis (Tick Level)',
    xaxis_title='Time',
    yaxis_title='Price',
    xaxis_rangeslider_visible=False,
    template='plotly_white', # Pro-looking simple theme
    width=1000,
    height=600
)

fig.show()